In [1]:
import torch
import os
import random
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from tx1_pre_process import *
from old_preprocess import *

In [2]:
DATA_PATH = '/home/ralleking/Downloads/nes_txt/'

In [3]:
def read_file(fname):
    with open(fname) as f:
        content = f.read().splitlines()
        content = [event.rsplit('_', 1) for event in content]
        content.append(['END', '0'])
    return content

def to_one_hot(vocab, event):
    one_hot = np.zeros(len(vocab))
    index = vocab[event]
    one_hot[index] = 1
    return one_hot

def from_one_hot(vocab, vector):
    ind = np.argmax(vector)
    for k, v in vocab.items():
        if v == ind:
            return k

In [4]:
songs = []
for filename in os.listdir(DATA_PATH):
    file = read_file(DATA_PATH + filename)
    songs.extend(file) 

In [5]:
df = pd.DataFrame(songs, columns=['event', 'time'])
df['note_on'] = df.time.apply(lambda x: x.isdigit())
df['time'] = pd.to_numeric(df['time'].replace('NOTEOFF',0))
df.loc[df.note_on == False, 'event'] = df.event.apply(lambda x: x + '_NOTEOFF')
df.loc[df.event == 'END_NOTEOFF', 'event'] = 'END'

In [6]:
vocab = {}
i = 0
for item in df['event'].unique():    
    vocab[item] = i
    i += 1
    
df['one_hot_x'] = df.event.apply(lambda x: to_one_hot(vocab, x))
df['label'] = df.one_hot_x.apply(lambda x: np.argmax(x))

In [7]:
data = df.values
df

,event,time,note_on,one_hot_x,label
0,WT,1,True,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,P2_NOTEON,84,True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,WT,3,True,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,P2_NOTEON,49,True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,WT,3,True,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
5,P1_NOTEON,77,True,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
6,WT,3,True,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
7,P1_NOTEON,56,True,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
8,WT,4383,True,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
9,P1_NOTEOFF,0,False,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3


In [25]:
start = 0
chunk_size = 10
data_pipe1 = []
data_pipe2 = []
data_pipe3 = []
num_songs = 0
for i in range(0, len(data)):    
    chunk_x = data[start:chunk_size]
    chunk_y = data[chunk_size:chunk_size+1]
    start = chunk_size + 1
    chunk_size = chunk_size + 11
    if np.in1d('END', chunk_x).all():
        num_songs +=1
        continue
    if np.in1d('END', chunk_y).all():
        num_songs +=1
        continue
        
    #Add correct columns to correct datapipes
    data_pipe1.append((chunk_x[:,3], chunk_y[0,4]))
    data_pipe2.append((chunk_x[:,3], chunk_y[:,2]))
    data_pipe3.append((chunk_x[:,[1,3]], chunk_y[:,1]))
    
    #For convenience
    if i>1000:
        break
print("Totalt number of songs " + str(num_songs))


Totalt number of songs 2


In [36]:
#Has only sequence information. That is x = 10 previous one hot enchodings of sequences, y = the next one
data1 = torch.utils.data.DataLoader(data_pipe1)

#Has sequence to note on/note off. x = 10 previous one hot and y = next note: on/off (bool)
data2 = torch.utils.data.DataLoader(data_pipe2)

#Has time for 10 previous timesteps in first column, then one hot seq in 2nd, y is the time for the following note
data3 = torch.utils.data.DataLoader(data_pipe3)

#idea is to predict first note on or note off, then which note, then for how long